## Preprocessing for Sarcasm Detection

This notebook performs the complete preprocessing pipeline for the Kosovo Albanian News dataset,
preparing the data for sarcasm detection modeling.

The preprocessing steps include:
- Loading the raw dataset
- Inspecting dataset structure and data types
- Analyzing and handling missing values
- Detecting and removing duplicate records
- Creating a unified text representation
- Performing basic text normalization
- Validating the final dataset
- Saving the preprocessed dataset for downstream modeling tasks

In [2]:
# Used Libraries

import pandas as pd
import os

In [3]:
# Helper methods

def print_dataset(text, df):
    print("\n" + text + ":")
    display(df.head())

def read_dataset(path):
    return pd.read_csv(path)

### Constants

In [4]:
df_columns = ['content', 'date', 'title', 'category', 'author', 'source']
df_path = "../data/kosovo_news.csv"
preprocessed_df_path = "../data/preprocessed_kosovo_news.csv"

### Read dataset

In [5]:
df = read_dataset(df_path)

### Data Overview

#### Dataset structure and data types

In [16]:
# Display dataset structure, data types and memory usage
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2593450 entries, 0 to 2593449
Data columns (total 6 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   content   object
 1   date      object
 2   title     object
 3   category  object
 4   author    object
 5   source    object
dtypes: object(6)
memory usage: 118.7+ MB


#### Missing Values Analysis

In [17]:
# Count missing values per column
df.isnull().sum()

content     10409
date            2
title           0
category    26551
author       8577
source          0
dtype: int64

#### Duplicate Records Analysis

In [18]:
# Count duplicate rows based on article content
duplicates_before = df.duplicated(subset=df_columns).sum()
print(f"Duplicate rows before removal: {duplicates_before}")

Duplicate rows before removal: 613866


### Data Preprocessing

#### Missing values handling

In [6]:
# Remove rows that contain null values in any of these columns
df = df.dropna(subset=df_columns)
df.isnull().sum()

content     0
date        0
title       0
category    0
author      0
source      0
dtype: int64

#### Duplicate removal

In [7]:
df = df.drop_duplicates(subset=df_columns, keep="first")

duplicates_after = df.duplicated(subset=df_columns).sum()
print(f"Duplicate rows after removal: {duplicates_after}")

Duplicate rows after removal: 0


#### Create unified text column

In [8]:
# Combine title and content into a single text column
df["text"] = (
    df["title"].astype(str) + " " + df["content"].astype(str)
).str.strip()

df.head()

,content,date,title,category,author,source,text
0,Kur Beatrice Borromeo u martua me Pierre Casir...,2021-08-26 12:24:57,As Kate as Meghan; ja cila është princesha më ...,Fun;Argëtim,Donika Gjoni,Lajmi,As Kate as Meghan; ja cila është princesha më ...
1,Gjatë kontrollit të automjetit zyrtarët polico...,2021-08-26 12:14:38,"I kapen 10 kg substanca narkotike në BMW X5, a...",Lajme;Nacionale,Flutura Toplana,Lajmi,"I kapen 10 kg substanca narkotike në BMW X5, a..."
2,Së fundmi Real Madrid dërgoi një ofertë 160 mi...,2021-08-26 12:14:00,"E fundit, Mbappe mund të zyrtarizohet nesër te...",La Liga;Lajme futbolli;Sport,Muhamet Mehmeti,Lajmi,"E fundit, Mbappe mund të zyrtarizohet nesër te..."
3,Enca Haxhia njihet si një ndër këngëtaret më s...,2021-08-26 12:13:58,Enca e quan jetë pushimin në plazh me poza në ...,nan;Entertainment,Fjolla Gashi,Lajmi,Enca e quan jetë pushimin në plazh me poza në ...
4,Gurët në veshka janë depozitime minerale që fo...,2021-08-26 12:07:45,Gurët në veshka – Kurat natyrale dhe si t’i pë...,Lifestyle;Shëndeti,Donika Gjoni,Lajmi,Gurët në veshka – Kurat natyrale dhe si t’i pë...


#### Drop unused columns

In [9]:
# Remove columns that are not used in modeling
df = df.drop(columns=["content", "date", "author"])

df.columns

Index(['title', 'category', 'source', 'text'], dtype='object')

#### Text normalization

In [11]:
# Convert text to lowercase
df["text"] = df["text"].str.lower()

# Remove URLs
df["text"] = df["text"].str.replace(r"http\S+|www\S+", "", regex=True)

# Remove extra whitespace
df["text"] = df["text"].str.replace(r"\s+", " ", regex=True)

df.head()

,title,category,source,text
0,As Kate as Meghan; ja cila është princesha më ...,Fun;Argëtim,Lajmi,as kate as meghan; ja cila është princesha më ...
1,"I kapen 10 kg substanca narkotike në BMW X5, a...",Lajme;Nacionale,Lajmi,"i kapen 10 kg substanca narkotike në bmw x5, a..."
2,"E fundit, Mbappe mund të zyrtarizohet nesër te...",La Liga;Lajme futbolli;Sport,Lajmi,"e fundit, mbappe mund të zyrtarizohet nesër te..."
3,Enca e quan jetë pushimin në plazh me poza në ...,nan;Entertainment,Lajmi,enca e quan jetë pushimin në plazh me poza në ...
4,Gurët në veshka – Kurat natyrale dhe si t’i pë...,Lifestyle;Shëndeti,Lajmi,gurët në veshka – kurat natyrale dhe si t’i pë...


#### Final dataset validation

In [12]:
# Final check for missing values
df.isnull().sum()

title       0
category    0
source      0
text        0
dtype: int64

In [13]:
# Final dataset shape
df.shape

(1941750, 4)

#### Save preprocessed dataset

In [14]:
os.makedirs(os.path.dirname(preprocessed_df_path), exist_ok=True)

df.to_csv(preprocessed_df_path, index=False)

print(f"Preprocessed dataset saved to: {preprocessed_df_path}")

print_dataset("Dataset info ->", df)

Preprocessed dataset saved to: ../data/preprocessed_kosovo_news.csv

Dataset info ->:


,title,category,source,text
0,As Kate as Meghan; ja cila është princesha më ...,Fun;Argëtim,Lajmi,as kate as meghan; ja cila është princesha më ...
1,"I kapen 10 kg substanca narkotike në BMW X5, a...",Lajme;Nacionale,Lajmi,"i kapen 10 kg substanca narkotike në bmw x5, a..."
2,"E fundit, Mbappe mund të zyrtarizohet nesër te...",La Liga;Lajme futbolli;Sport,Lajmi,"e fundit, mbappe mund të zyrtarizohet nesër te..."
3,Enca e quan jetë pushimin në plazh me poza në ...,nan;Entertainment,Lajmi,enca e quan jetë pushimin në plazh me poza në ...
4,Gurët në veshka – Kurat natyrale dhe si t’i pë...,Lifestyle;Shëndeti,Lajmi,gurët në veshka – kurat natyrale dhe si t’i pë...
